# How to paraphrase text using transformers in Python

Notes and References:

PEGASUS is an acronym for Pre-training with Extracted Gap-sentences for Abstractive SUmmarization Sequence-to-sequence models
PEGASUS model used herein is from Huggingface's transformers library
PEGASUS model from Google Research
Read the original paper PEGASUS: Pre-training with Extracted Gap-sentences for Abstractive Summarization

# Install Libraries

In [12]:
!pip install sentence-splitter
!pip install transformers
!pip install SentencePiece
!pip install flask transformers


# Download pretrained Google Pegasus Paraphrase Model and its tokenizer

In [2]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

In [3]:
model = PegasusForConditionalGeneration.from_pretrained('tuner007/pegasus_paraphrase')
tokenizer = PegasusTokenizer.from_pretrained('tuner007/pegasus_paraphrase')

config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at tuner007/pegasus_paraphrase and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

# Tokenization

In [13]:
text = "The ultimate test of your knowledge is your capacity to convey it to another."

batch = tokenizer([text], padding=True, truncation=True, max_length=60, return_tensors='pt')
output = model.generate(**batch, max_length=60, num_beams=5, num_return_sequences=5, temperature=1.5, do_sample=True)
# output = model.generate(**batch, max_length=60, num_beams=5, num_return_sequences=5, do_sample=True)

In [9]:
results = tokenizer.batch_decode(output, skip_special_tokens=True)
results

['The test of your knowledge is your ability to convey it.',
 'The ability to convey your knowledge is the ultimate test of your knowledge.',
 'The test of your knowledge is your ability to communicate it.',
 'Your capacity to convey your knowledge is the ultimate test of your knowledge.',
 'The test of your knowledge is how well you can convey it.']

# Predictive System (Generate Paraphrase)

In [10]:
def get_response(input_text,num_return_sequences,num_beams):
  batch = tokenizer([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt")
  translated = model.generate(**batch,max_length=60,num_beams=num_beams, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

In [11]:
num_beams = 10
num_return_sequences = 10
context = "The ultimate test of your knowledge is your capacity to convey it to another."
get_response(context,num_return_sequences,num_beams)

['The test of your knowledge is your ability to convey it.',
 'The ability to convey your knowledge is the ultimate test of your knowledge.',
 'The ability to convey your knowledge is the most important test of your knowledge.',
 'Your capacity to convey your knowledge is the ultimate test of it.',
 'The test of your knowledge is your ability to communicate it.',
 'Your capacity to convey your knowledge is the ultimate test of your knowledge.',
 'Your capacity to convey your knowledge to another is the ultimate test of your knowledge.',
 'Your capacity to convey your knowledge is the most important test of your knowledge.',
 'The test of your knowledge is how well you can convey it.',
 'Your capacity to convey your knowledge is the ultimate test.']